Explore different trajectory parameterization options.

Step 1 is how to parameterize a continuous 2D $(x(t),y(t))$ trajectory. 
We can use a polynomial, a spline, or a neural network.

Step 2 is to use differential flatness to obtain the full state and control trajectories.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# autoreload
%load_ext autoreload
%autoreload 2

# Step 1: Trajectory parameterization
---

## Polynomial

In [ ]:
class PolyPath:
    def __init__(self, d, n, start, end, free_coeffs=None):
        """
        Constrain endpoints, zero velocity at endpoints.

        d : int
            Dimension of the path
        n : int
            Degree of the polynomial
        free_coeffs : np.ndarray (n+1-4, d)
            Free coefficients. The first 4 coefficients are constrained
            
        """
        self.d = d
        self.n = n
        self.exponents = torch.arange(n+1, device=device).float()
        
        # Compute full coefficients
        self.coeffs = torch.zeros((n+1, d), device=device)
        self.coeffs[4:] = free_coeffs
        self.coeffs[3] = 2 * (start - end) - free_coeffs.T @ torch.arange(2, n-1, device=device).float()
        self.coeffs[2] = (end - start) - self.coeffs[3] - free_coeffs.T @ torch.ones(n-3, device=device)
        self.coeffs[1] = 0.0
        self.coeffs[0] = start
        

    def eval(self, t):
        N = len(t)
        T = t[:, None] ** self.exponents
        return T @ self.coeffs

In [ ]:
d = 2 # dim
n = 8 # degree

for i in range(100):
    free_coeffs = np.random.rand(n+1-4, d)
    start = np.array([0,0])
    end = np.array([1,1])
    path = PolyPath(d, n, start, end, free_coeffs)

    t = np.linspace(0, 1, 100)
    X = path.eval(t)
    plt.plot(X[:, 0], X[:, 1])

In [ ]:
# Optimize a poly path
d = 2
n = 10
start = torch.tensor([0.0, 0.0], device=device)
end = torch.tensor([1.0 ,1.0], device=device)

free_coeffs = torch.rand(n+1-4, d, device=device, requires_grad=True)
path = PolyPath(d, n, start, end, free_coeffs)

# Optimize to avoid circle at (0.5, 0.5)

def loss(X):
    # distance loss
    return torch.diff(X, dim=0).norm(dim=1).sum()

def loss_1(X):
    return torch.sum(1.0 / (1e-7 + torch.norm(X - torch.tensor([0.25, 0.25], device=device), dim=1)))

def loss_2(X):
    return torch.sum(1.0 / (1e-7 + torch.norm(X - torch.tensor([0.75, 0.75], device=device), dim=1)))

optimizer = torch.optim.Adam([free_coeffs], lr=1e-2)
for i in range(1000):
    optimizer.zero_grad()
    path = PolyPath(d, n, start, end, free_coeffs)
    X = path.eval(t)
    l = loss(X) + 1e-2*loss_1(X) + 1e-2*loss_2(X)
    l.backward()
    optimizer.step()
    if i % 100 == 0:
        print(l.item())

In [ ]:
# free_coeffs = torch.rand(n+1-4, d, device=device, requires_grad=True)
path = PolyPath(d, n, start, end, free_coeffs)
t = torch.linspace(0, 1, 100, device=device)
X = path.eval(t)
plt.plot(X[:, 0].detach().cpu().numpy(), X[:, 1].detach().cpu().numpy())

## B-spline

In [ ]:
import scipy.interpolate as spi

In [ ]:
start_point = np.array([0, 0])
end_point = np.array([1, 1])

control_points = np.array([
    [0.1, 0.1],
    [0.2, 0.5],
    [0.7, 0.6]
])

In [ ]:
points = np.vstack([start_point, control_points, end_point])
x = points[:, 0]
y = points[:, 1]

# Fit the B-spline
tck, u = spi.splprep([x, y], s=0)

# Evaluate the spline
unew = np.linspace(0, 1.0, 1000)
out = spi.splev(unew, tck)

# Plot the results
plt.figure()
plt.plot(x, y, 'ro', label='Control Points')
plt.plot(out[0], out[1], 'b-', label='B-spline Curve')
plt.legend()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('B-spline fitting with start and end points')
plt.show()

# Step 2: Differential Flatness
---

## Standard Dubin's car